In [1]:
# coding: utf-8

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os
import csv

#BATCH_SIZE = 100
#WEIGHT_DECAY = 0.00005
#LEARNING_RATE = 0.0001
#EPOCH = 100
PATH = os.getcwd()
#TRAINPATH = "C:\\Users\\arimoto\\cnntest\\train"
#TESTPATH = "C:\\Users\\arimoto\\cnntest\\test"
#VALIDPATH = "C:\\Users\\arimoto\\cnntest\\valid"

#手指姿勢CSVファイル一覧取得
filelists = glob.glob(PATH + "\\dataset\\*.csv")
#print(filelists)

data_sets = []
for i in range(len(filelists)):
    data_points = []
    data_masks = []
    invcnt = 0
    with open(filelists[i]) as f:
        reader = csv.reader(f)
        num = 0
        for row in reader:
            if num == 0:
                for point in row:
                    data_points.append(int(point))
                    if int(point) == -10000:
                        invcnt += 1
                        data_masks.append(0)
                    else:
                        data_masks.append(1)
                data_points = np.asarray(data_points)
                data_masks = np.asarray(data_masks)
                num += 1
    if invcnt != 63:
        data_pair = []
        data_pair.append(data_points)
        data_pair.append(data_masks)
        data_sets.append(data_pair)

print("fin loading")
data_sets = np.array(data_sets)
#img_arrays = np.array(img_arrays)
print("data" + str(data_sets[0]))
print(type(data_sets[0][0]))
#print(data_sets.shape())

fin loading
data[[ 29   2  27  11  22  19  15  25   7  29  -2  29 -11  27 -19  22 -25  15
  -29   7 -29  -2 -27 -11 -22 -19 -15 -25  -7 -29   2 -29  11 -27  19 -22
   25 -15  29  -7]
 [  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1]]
<class 'numpy.ndarray'>


In [2]:
# trainデータとtestデータに分割
data_sets_train, data_sets_test, label_sets_train, label_sets_test = train_test_split(
    data_sets,
    data_sets,
    random_state = 0,
    test_size = 0.2
)
print(data_sets_train.shape, label_sets_train.shape, data_sets_test.shape, label_sets_test.shape)

(960, 2, 40) (960, 2, 40) (240, 2, 40) (240, 2, 40)


In [3]:
data_train = []
mask_train = []
data_test = []
mask_test = []
label_train = []
lmask_train = []
label_test = []
lmask_test = []
for i in range(len(data_sets_train)):
    data_train.append(data_sets_train[i][0])
    mask_train.append(data_sets_train[i][1])
    label_train.append(label_sets_train[i][0])
    lmask_train.append(label_sets_train[i][1])
for i in range(len(data_sets_test)):
    data_test.append(data_sets_test[i][0])
    mask_test.append(data_sets_test[i][1])
    label_test.append(label_sets_test[i][0])
    lmask_test.append(label_sets_test[i][1])
    

print(data_train[0])
print(mask_train[0])
print(label_train[0])
print(lmask_train[0])

[ 26  14  20  21  13  27   4  29  -5  29 -14  26 -21  20 -27  13 -29   4
 -29  -5 -26 -14 -20 -21 -13 -27  -4 -29   5 -29  14 -26  21 -20  27 -13
  29  -4  29   5]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]
[ 26  14  20  21  13  27   4  29  -5  29 -14  26 -21  20 -27  13 -29   4
 -29  -5 -26 -14 -20 -21 -13 -27  -4 -29   5 -29  14 -26  21 -20  27 -13
  29  -4  29   5]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [4]:
data_trainF = np.array(data_train).astype('float32')#/100
data_testF = np.array(data_test).astype('float32')#/100
mask_trainF = np.array(mask_train).astype('float32')
mask_testF = np.array(mask_test).astype('float32')
label_trainF = np.array(label_train).astype('float32')#/100
label_testF = np.array(label_test).astype('float32')#/100
lmask_trainF = np.array(lmask_train).astype('float32')
lmask_testF = np.array(lmask_test).astype('float32')
print("data        train ",data_trainF.shape)
print("data        test  ",data_testF.shape)
print("data-mask   train ",mask_trainF.shape)
print("data-mask   test  ",mask_testF.shape)
print("label       train ",label_trainF.shape)
print("label       test  ",label_testF.shape)
print("label-mask  train ",lmask_trainF.shape)
print("label-mask  test  ",lmask_testF.shape)

data        train  (960, 40)
data        test   (240, 40)
data-mask   train  (960, 40)
data-mask   test   (240, 40)
label       train  (960, 40)
label       test   (240, 40)
label-mask  train  (960, 40)
label-mask  test   (240, 40)


In [5]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, datas, labels, data_masks, label_masks, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.label = labels
        self.masks = data_masks
        self.lmasks = label_masks
        
        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        #batch_set = []
        
        i_data = self.data[idx]
        i_label = self.label[idx]
        i_mask = self.masks[idx]
        i_lmask = self.lmasks[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_label = np.array(i_label.astype(np.float32))
        out_data = np.array(i_data.astype(np.float32))
        out_mask = np.array(i_mask.astype(np.float32))
        out_lmask = np.array(i_lmask.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        #batch_set.append(out_data)
        #batch_set.append(out_label)
        #batch_set.append(out_img)
        #batch_set.append(out_mask)
        #batch_set.append(out_lmask)
        #print(out_data.shape)
        #print(out_label.shape)
        #print(out_mask.shape)
        #print(out_lmask.shape)
        
        return out_data, out_label, out_mask, out_lmask
        #return batch_set

In [6]:
trainset = Mydatasets(datas = data_trainF, labels = label_trainF, data_masks = mask_trainF, label_masks = lmask_trainF)
testset = Mydatasets(datas = data_testF, labels = label_testF, data_masks = mask_testF, label_masks = lmask_testF)

In [7]:
BATCH_SIZE = 120
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

In [8]:
class Ignore(nn.Module):
    def __init__(self, input_dim, output_dim, activation = lambda x: x):
        '''
        引数:
            input_dim: 入力次元
            output_dim: 出力次元
            activation: 活性化関数
        パラメータ:
            W: 重み
            b: バイアス
        '''
        super().__init__()
        self.W = nn.Parameter(torch.Tensor(np.random.normal(size = (output_dim, input_dim))))
        self.b = nn.Parameter(torch.Tensor(np.zeros(output_dim)))
        self.activation = activation
    def forward(self, x, mask):
        
        out = torch.empty_like(x).to(torch.device("cuda:0"))
        
        masked_x = torch.mul(x, mask)
        
        try:
            m_size = torch.Tensor(mask.size()[1]).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask, 1).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            for b in range(out.size()[0]):
                out[b] = torch.add(torch.mul(rate[b], torch.sum(torch.mul(masked_x[b], self.W), 1)), self.b)
        except IndexError:
            m_size = torch.Tensor(mask.size()).to(torch.device("cuda:0"))
            m_sum = torch.sum(mask).to(torch.device("cuda:0"))
            rate = (m_size.size()[0] / m_sum).to(torch.device("cuda:0"))
            out = torch.add(torch.mul(rate, torch.sum(torch.mul(masked_x, self.W))), self.b)
        
        return self.activation(out)

In [9]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.mask = Ignore(40, 40)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.dense_enc1 = nn.Linear(40, 30)
        self.bn1 = nn.BatchNorm1d(30)
        self.dense_enc2 = nn.Linear(30, 20)
        self.bn2 = nn.BatchNorm1d(20)
        self.dense_enc3 = nn.Linear(20,10)
    
        self.dense_dec1 = nn.Linear(10,20)
        self.bn4 = nn.BatchNorm1d(20)
        self.dense_dec2 = nn.Linear(20, 30)
        self.bn5 = nn.BatchNorm1d(30)
        self.drop1 = nn.Dropout(p=0.2)
        self.dense_dec3 = nn.Linear(30, 40)

    def encoder(self, x, m):
        #x = torch.div(x, 100.)
        #x = self.mask(x, m)
        x = self.dense_enc1(x)
        x = self.relu(self.bn1(x))
        x = self.dense_enc2(x)
        x = self.relu(self.bn2(x))
        x = self.dense_enc3(x)
        return x

    def decoder(self, x, m):
        x = self.dense_dec1(x)
        x = self.relu(self.bn4(x))
        x = self.dense_dec2(x)
        x = self.relu(self.bn5(x))
        #x = self.drop1(x)
        x = self.dense_dec3(x)
        #x = self.mask(x, m)
        #x = torch.mul(x, 100.)
        return x

    def forward(self, x, m):
        z = self.encoder(x, m)
        x = self.decoder(z, m)
        return x, z

In [10]:
class MaskMSELoss(nn.Module):

    def __init__(self):
        super(MaskMSELoss, self).__init__()
        #self.margin = margin

    def forward(self, inputs, labels, lmasks):
        m_size = lmasks.size()[1]
        m_sum = torch.sum(lmasks, 1)
        rate = (m_size / m_sum).reshape(BATCH_SIZE,-1)
        for i in range(rate.size()[0]):
            if np.isinf(rate[i].item()):
                rate[i] = 0.
        masked_in = torch.mul(inputs, lmasks)
        masked_lb = torch.mul(labels, lmasks)
        
        #loss = torch.sub(masked_in, masked_lb)
        #loss = torch.pow(loss, 2)
        #loss = torch.sum(loss, 1)
        #loss = torch.mul(rate, loss)
        #loss = torch.mean(loss)
        
        loss = torch.sub(masked_in, masked_lb)
        loss = torch.pow(loss, 2)
        loss = torch.mul(loss, rate)
        loss = torch.mean(loss, 1)
        loss = torch.mean(loss)
        
        return loss

In [11]:
LEARNING_RATE = 0.01
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9
#EPOCH = 100
device = torch.device("cuda:0")
net = Autoencoder()
net = net.to(device)
criterion = MaskMSELoss()
#criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
#train_acc_value=[]       #trainingのaccuracyを保持するlist
test_loss_value=[]       #testのlossを保持するlist
#test_acc_value=[]        #testのaccuracyを保持するlist 
train_mid_out_value = [] #trainingの中間層の出力を保持するlist
test_mid_out_value = []  #testの中間層の出力を保持するlist
train_input_value = []   #trainingの入力を保持するlist
test_input_value = []    #testの入力を保持するlist
train_output_value = []  #trainingの出力を保持するlist
test_output_value = []   #testの出力を保持するlist

train_size = data_trainF.shape[0]
test_size = data_testF.shape[0] 

max_train_loss_value = 0.
max_test_loss_value = 0.

In [12]:
EPOCH = 1000
for epoch in range(EPOCH):#EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    num = 0
    for (inputs, labels, input_mask, label_mask) in trainloader:
        num += 1
        print(num)
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        #print("input(train): ", inputs.size())
        outputs, middle = net(inputs, input_mask)
        #print("input(train): ", inputs[0], inputs[1])
        #print("middle(train): ", middle[0], middle[1])
        #print("output(train): ", outputs[0], outputs[1])
        loss = criterion(outputs, labels, label_mask)#crit_outputs, crit_labels)
        ##loss = criterion(outputs, labels)
        ##print("loss(train): ", loss)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    #sum_correct = 0         #正解率の合計
    sum_total = 0           #dataの数の合計

    print("test_train")
    ttrain = 0
    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels, input_mask, label_mask) in trainloader:
        ttrain += 1
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        outputs, middle = net(inputs, input_mask)
        if ttrain == int(train_size / BATCH_SIZE):
            ##print("input(train_test): ", inputs[0], inputs[1])
            ##print("middle(train_test): ", middle[0], middle[1])
            ##print("output(train_test): ", outputs[0], outputs[1])
            train_input_value.append(inputs)
            train_mid_out_value.append(middle)
            train_output_value.append(outputs)
        loss = criterion(outputs, labels, label_mask)
        ##loss = criterion(outputs, labels)
        ##print("loss(train_test): ", loss)
        sum_loss += loss.item()                            #lossを足していく
        #_, predicted = outputs.max(1)                      #出力の最大値の添字(予想位置)を取得
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
        #sum_correct += (predicted == labels).sum().item()  #予想位置と実際の正解を比べ,正解している数だけ足す
        
    print("len train dataset: ", len(trainloader.dataset))
    print("train mean loss={}".format(sum_loss*BATCH_SIZE/len(trainloader.dataset)))  #loss出力
    train_loss_value.append(sum_loss*BATCH_SIZE/len(trainloader.dataset))  #traindataのlossをグラフ描画のためにlistに保持
    #train_acc_value.append(float(sum_correct/sum_total))   #traindataのaccuracyをグラフ描画のためにlistに保持
    if sum_loss*BATCH_SIZE/len(trainloader.dataset) > max_train_loss_value:
        max_train_loss_value = sum_loss*BATCH_SIZE/len(trainloader.dataset)

    sum_loss = 0.0
    #sum_correct = 0
    sum_total = 0

    print("test_test")
    ttest = 0
    #test dataを使ってテストをする
    for (inputs, labels, input_mask, label_mask) in testloader:
        ttest += 1
        inputs, labels, input_mask, label_mask = \
        inputs.to(device), labels.to(device), input_mask.to(device), label_mask.to(device)
        optimizer.zero_grad()
        outputs, middle = net(inputs, input_mask)
        if ttest == int(test_size / BATCH_SIZE):
            ##print("input(test_test): ", inputs[0], inputs[1])
            ##print("middle(test_test): ", middle[0], middle[1])
            ##print("output(test_test): ", outputs[0], outputs[1])
            test_input_value.append(inputs)
            test_mid_out_value.append(middle)
            test_output_value.append(outputs)
        loss = criterion(outputs, labels, label_mask)
        ##loss = criterion(outputs, labels)
        ##print("loss(test_test): ", loss)
        sum_loss += loss.item()
        #_, predicted = outputs.max(1)
        sum_total += labels.size(0)
        #sum_correct += (predicted == labels).sum().item()
    print("len test dataset: ", len(testloader.dataset))
    print("test mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(testloader.dataset)))
    test_loss_value.append(sum_loss*BATCH_SIZE/len(testloader.dataset))
    #test_acc_value.append(float(sum_correct/sum_total))
    if sum_loss*BATCH_SIZE/len(testloader.dataset) > max_test_loss_value:
        max_test_loss_value = sum_loss*BATCH_SIZE/len(testloader.dataset)

epoch 1
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=336.0605926513672
test_test
len test dataset:  240
test mean loss=335.57334899902344
epoch 2
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=6.210114538669586
test_test
len test dataset:  240
test mean loss=5.468377351760864
epoch 3
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=5.514724969863892
test_test
len test dataset:  240
test mean loss=5.382380485534668
epoch 4
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=19.150274515151978
test_test
len test dataset:  240
test mean loss=19.42509937286377
epoch 5
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=10.463879466056824
test_test
len test dataset:  240
test mean loss=10.39693021774292
epoch 6
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=4.442602574825287
test_test
len test dataset:  240
test mean loss=4.0291643142700195
epoch 7
1
2
3
4
5
6
7
8
test_train
len tra

test_test
len test dataset:  240
test mean loss=0.1757388599216938
epoch 102
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1961107300594449
test_test
len test dataset:  240
test mean loss=0.1631874404847622
epoch 103
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.17018322367221117
test_test
len test dataset:  240
test mean loss=0.2663465142250061
epoch 104
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.2876018099486828
test_test
len test dataset:  240
test mean loss=0.1764489896595478
epoch 105
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.21036131773144007
test_test
len test dataset:  240
test mean loss=0.16060684993863106
epoch 106
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.27182440273463726
test_test
len test dataset:  240
test mean loss=0.2721827030181885
epoch 107
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.19222850631922483
test_test
len tes

len train dataset:  960
train mean loss=0.17773104179650545
test_test
len test dataset:  240
test mean loss=0.27535415440797806
epoch 202
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.12496946938335896
test_test
len test dataset:  240
test mean loss=0.17429231852293015
epoch 203
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1693606898188591
test_test
len test dataset:  240
test mean loss=0.13836165145039558
epoch 204
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.2005866076797247
test_test
len test dataset:  240
test mean loss=0.20011326298117638
epoch 205
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.2129477895796299
test_test
len test dataset:  240
test mean loss=0.19026214629411697
epoch 206
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.2186673991382122
test_test
len test dataset:  240
test mean loss=0.19095386937260628
epoch 207
1
2
3
4
5
6
7
8
test_train
len train data

test mean loss=0.1792006902396679
epoch 301
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.16966160107403994
test_test
len test dataset:  240
test mean loss=0.1732686311006546
epoch 302
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.15926005598157644
test_test
len test dataset:  240
test mean loss=0.20019600167870522
epoch 303
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.20984907913953066
test_test
len test dataset:  240
test mean loss=0.12847566232085228
epoch 304
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.14121691975742579
test_test
len test dataset:  240
test mean loss=0.17717212066054344
epoch 305
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.23213698901236057
test_test
len test dataset:  240
test mean loss=0.16402532532811165
epoch 306
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.20050465036183596
test_test
len test dataset:  240
test mean lo

train mean loss=0.16721624042838812
test_test
len test dataset:  240
test mean loss=0.15241721644997597
epoch 401
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.17612006608396769
test_test
len test dataset:  240
test mean loss=0.1693177931010723
epoch 402
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1752489022910595
test_test
len test dataset:  240
test mean loss=0.12132195755839348
epoch 403
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.17124176118522882
test_test
len test dataset:  240
test mean loss=0.2075733244419098
epoch 404
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.16866084840148687
test_test
len test dataset:  240
test mean loss=0.15315894782543182
epoch 405
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.15257826261222363
test_test
len test dataset:  240
test mean loss=0.24462395161390305
epoch 406
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean lo

...

test_test
len test dataset:  240
test mean loss=0.13470225036144257
epoch 600
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1171990605071187
test_test
len test dataset:  240
test mean loss=0.15685079246759415
epoch 601
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13728644791990519
test_test
len test dataset:  240
test mean loss=0.17583002895116806
epoch 602
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.12213785201311111
test_test
len test dataset:  240
test mean loss=0.12197743356227875
epoch 603
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.22176532819867134
test_test
len test dataset:  240
test mean loss=0.1329466812312603
epoch 604
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13208485580980778
test_test
len test dataset:  240
test mean loss=0.1351245641708374
epoch 605
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.15046417526900768
test_test
len

test_train
len train dataset:  960
train mean loss=0.12799251452088356
test_test
len test dataset:  240
test mean loss=0.1630857065320015
epoch 700
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11953355930745602
test_test
len test dataset:  240
test mean loss=0.14253754541277885
epoch 701
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.24171881843358278
test_test
len test dataset:  240
test mean loss=0.12579847127199173
epoch 702
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11465802323073149
test_test
len test dataset:  240
test mean loss=0.1254609115421772
epoch 703
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.09579719696193933
test_test
len test dataset:  240
test mean loss=0.14019066467881203
epoch 704
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1583635350689292
test_test
len test dataset:  240
test mean loss=0.18242895975708961
epoch 705
1
2
3
4
5
6
7
8
test_train
le

train mean loss=0.11124477256089449
test_test
len test dataset:  240
test mean loss=0.14534585177898407
epoch 799
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.159500683657825
test_test
len test dataset:  240
test mean loss=0.18762053176760674
epoch 800
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11377233825623989
test_test
len test dataset:  240
test mean loss=0.12116577476263046
epoch 801
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.16113140061497688
test_test
len test dataset:  240
test mean loss=0.1663660667836666
epoch 802
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.178298844024539
test_test
len test dataset:  240
test mean loss=0.1408606916666031
epoch 803
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.12240864429622889
test_test
len test dataset:  240
test mean loss=0.1695684753358364
epoch 804
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0

test mean loss=0.17887390404939651
epoch 898
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.14697536639869213
test_test
len test dataset:  240
test mean loss=0.1026170626282692
epoch 899
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11407375242561102
test_test
len test dataset:  240
test mean loss=0.10992057248950005
epoch 900
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.19678229838609695
test_test
len test dataset:  240
test mean loss=0.1399466097354889
epoch 901
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13606043066829443
test_test
len test dataset:  240
test mean loss=0.1353662572801113
epoch 902
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.1313421269878745
test_test
len test dataset:  240
test mean loss=0.11835497617721558
epoch 903
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11852836702018976
test_test
len test dataset:  240
test mean loss

test_test
len test dataset:  240
test mean loss=0.11228552833199501
epoch 948
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.09924287535250187
test_test
len test dataset:  240
test mean loss=0.1355041228234768
epoch 949
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.18191285338252783
test_test
len test dataset:  240
test mean loss=0.14456526935100555
epoch 950
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13219129852950573
test_test
len test dataset:  240
test mean loss=0.1377728134393692
epoch 951
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13297037687152624
test_test
len test dataset:  240
test mean loss=0.11358149349689484
epoch 952
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.13788016512989998
test_test
len test dataset:  240
test mean loss=0.15746000409126282
epoch 953
1
2
3
4
5
6
7
8
test_train
len train dataset:  960
train mean loss=0.11462468933314085
test_test
le

In [15]:
plt.figure(figsize=(6,6))      #グラフ描画用
ylim = max(max_train_loss_value, max_test_loss_value)

baseEPOCH = 100
num_epoch = 10 #100単位
act_num_epoch = baseEPOCH * num_epoch

#以下グラフ描画
plt.plot(range(act_num_epoch), train_loss_value)
plt.plot(range(act_num_epoch), test_loss_value, c='#00ff00')
plt.xlim(0, act_num_epoch)
#plt.ylim(0, ylim)
plt.ylim(0, 10)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig(PATH + "\\loss_image_" + str(num_epoch) + "2.png")
plt.clf()

<Figure size 432x432 with 0 Axes>

In [14]:
#結果画像出力
target_epoch = 999

fig = plt.figure(figsize=(6,6))
for i in range(len(train_input_value[target_epoch])):
    input_x = []
    input_y = []
    mid = []
    output_x = []
    output_y = []
    connect_x = []
    connect_y = []
    ax = fig.add_subplot(1,1,1)
    for j in range(int(len(train_input_value[target_epoch][0])/2)):
        input_x.append(train_input_value[target_epoch][i][j*2+0].item())
        input_y.append(train_input_value[target_epoch][i][j*2+1].item())
        output_x.append(train_output_value[target_epoch][i][j*2+0].item())
        output_y.append(train_output_value[target_epoch][i][j*2+1].item())
    #for k in range(len(input_x)):
    #    ax.scatter(input_x[k], input_y[k], c=(0.1+(k*0.04),1-(k*0.02),0.2+(k*0.03)), label = 'p'+str(k))
    #    ax.scatter(output_x[k], output_y[k], c=(0.1+(k*0.04),1-(k*0.02),0.2+(k*0.03)), label = 'p'+str(k))
    for l in range(len(input_x)):
        connect_x = [input_x[l], output_x[l]]
        connect_y = [input_y[l], output_y[l]]
        ax.plot(connect_x,connect_y,linestyle='solid',color='gray')
    ax.scatter(input_x, input_y, c='red', label = 'input')
    ax.scatter(output_x, output_y, c='blue', label = 'output')
    ax.set_title('input & output(train data, ' + str(target_epoch+1) + 'th epoch)')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\IOcomp_train_" + str(target_epoch) + "th_epoch_" + str(i) + ".png")
    fig.clf()
for i in range(len(test_input_value[target_epoch])):
    input_x = []
    input_y = []
    mid = []
    output_x = []
    output_y = []
    connect_x = []
    connect_y = []
    ax = fig.add_subplot(1,1,1)
    for j in range(int(len(test_input_value[target_epoch][0])/2)):
        input_x.append(test_input_value[target_epoch][i][j*2+0].item())
        input_y.append(test_input_value[target_epoch][i][j*2+1].item())
        output_x.append(test_output_value[target_epoch][i][j*2+0].item())
        output_y.append(test_output_value[target_epoch][i][j*2+1].item())
    #for k in range(len(input_x)):
    #    ax.scatter(input_x[k], input_y[k], c=(0.1+(k*0.04),1-(k*0.02),0.2+(k*0.03)), label = 'p'+str(k))
    #    ax.scatter(output_x[k], output_y[k], c=(0.1+(k*0.04),1-(k*0.02),0.2+(k*0.03)), label = 'p'+str(k))
    for l in range(len(input_x)):
        connect_x = [input_x[l], output_x[l]]
        connect_y = [input_y[l], output_y[l]]
        ax.plot(connect_x,connect_y,linestyle='solid',color='gray')    
    ax.scatter(input_x, input_y, c='red', label = 'input')
    ax.scatter(output_x, output_y, c='blue', label = 'output')
    ax.set_title('input & output(test data, ' + str(target_epoch+1) + 'th epoch)')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    #ax.legend(loc='upper right')
    fig.savefig(PATH + "\\IOcomp_test_" + str(target_epoch) + "th_epoch_" + str(i) + ".png")
    fig.clf()
plt.close()

In [ ]:
#出力値保存

with open(PATH + "\\network_outs.csv", mode='w') as f:
    writer = csv.writer(f)
    writer.writerow(["train_test"," "])
    for i in range(len(train_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(train_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            output_x = []
            output_y = []
            middle = []
            for k in range(int(len(train_input_value[i][j])/2)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(train_input_value[i][j][k*2+0].item())
                input_y.append(train_input_value[i][j][k*2+1].item())
                output_x.append(train_output_value[i][j][k*2+0].item())
                output_y.append(train_output_value[i][j][k*2+1].item())
            for l in range(len(train_mid_out_value[i][j])):
                middle.append(train_mid_out_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(middle)
    writer.writerow(["test_test"," "])
    for i in range(len(test_input_value)):                      #epoch数 loop 1000
        writer.writerow(["epoch:" + str(i+1)," "])
        for j in range(len(test_input_value[i])):               #batch size数 loop 10
            writer.writerow(["batch:" + str(j+1)," "])
            input_x = []
            input_y = []
            output_x = []
            output_y = []
            middle = []
            for k in range(int(len(test_input_value[i][j])/2)): #data size / 2 loop (x, y分離) 21(42)
                input_x.append(test_input_value[i][j][k*2+0].item())
                input_y.append(test_input_value[i][j][k*2+1].item())
                output_x.append(test_output_value[i][j][k*2+0].item())
                output_y.append(test_output_value[i][j][k*2+1].item())
            for l in range(len(test_mid_out_value[i][j])):
                middle.append(test_mid_out_value[i][j][l].item())
            writer.writerow(input_x)
            writer.writerow(input_y)
            writer.writerow(output_x)
            writer.writerow(output_y)
            writer.writerow(middle)
    print("fin save.")

In [22]:
torch.save(net.state_dict(), PATH + "\\model")

In [15]:
test = [-21,-20,24,16,-8,28,13,-26,-27,12,14,26,-15,-25,3,29,29,-6,-23,-19,4,29,-29,5,13,-26,6,-29,-27,12,15,25,-3,-29,-26,13,-27,-11,1,29,-20,-21]
testx2 = []
testx2.append(test)
testx2.append(test)
test = np.array(testx2).astype('float32')#/100
test_m = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
test_mx2 = []
test_mx2.append(test_m)
test_mx2.append(test_m)
test_m = np.array(test_mx2).astype('float32')
x = torch.from_numpy(test).clone().to(device)
m = torch.from_numpy(test_m).clone().to(device)
print("x: ", x[0], x.size())
print("m: ", m[0], m.size())

out, mid = net(x, m)

print("mid: ", mid[0], mid.size())
print("out: ", out[0], out.size())

x:  tensor([-21., -20.,  24.,  16.,  -8.,  28.,  13., -26., -27.,  12.,  14.,  26.,
        -15., -25.,   3.,  29.,  29.,  -6., -23., -19.,   4.,  29., -29.,   5.,
         13., -26.,   6., -29., -27.,  12.,  15.,  25.,  -3., -29., -26.,  13.,
        -27., -11.,   1.,  29., -20., -21.], device='cuda:0') torch.Size([2, 42])
m:  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.], device='cuda:0') torch.Size([2, 42])
mid:  tensor([ 0.1550,  0.5827, -1.4886,  1.0199,  0.8108,  0.2974,  1.4050,  1.3957,
         2.0596, -0.9116, -0.0410,  0.5965,  0.6650,  0.2561, -0.0219, -1.8135,
         0.0806, -1.6748,  0.4624,  1.7928,  1.0994, -2.0165, -1.7630, -0.5186],
       device='cuda:0', grad_fn=<SelectBackward>) torch.Size([2, 24])
out:  tensor([-0.1120,  0.8487,  0.8144,  2.0411, -1.7900,  4.5704,  3.8873,  6.6932,
         0.9492, -1.0785, -0.8894,  1